In [1]:
import numpy as np
import tensorflow as tf
import tensorflow 
import time
#pip install -U scikit-learn scipy matplotlib
#from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
import re

from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
#from tensorflow_model_optimization.sparsity import keras as sparsity

In [2]:
def load_model(m):
    model = tf.keras.models.load_model(m)
    return model

In [ ]:
import os
def prepare_dataset():
    # Load the training data into a tf.data.Dataset
    train_images_dir = "train2017"

    def load_images(images_dir=train_images_dir):
        images = []
        labels = []
        for image_filename in os.listdir(images_dir):
            # Load the image
            image = tf.io.read_file(os.path.join(images_dir, image_filename))
            image = tf.image.decode_jpeg(image, channels=3)

            # Load the label
            #label = # code to load label from file

            images.append(image)
            #labels.append(label)
        return images
        #, labels

    images = load_images(train_images_dir)
    train_dataset = tf.data.Dataset.from_tensor_slices((images))
    # Generate a representative dataset
    return train_dataset

def gen_rep():
    train_dataset=prepare_dataset()    
    representative_dataset = train_dataset.take(100).batch(1)
    return representative_dataset

In [3]:
quantize_layer_names1 = ['dense_1']
def just_convert_model_1(model,name='just_converted_quant_int_model1.tflite'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_quant_int_model = converter.convert()
    #saving converted model in "converted_quant_model.tflite" file
    open(name, "wb").write(tflite_quant_int_model)
    return tflite_quant_int_model
    

In [5]:
'''for layer_name in quantize_layer_names:
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    input_arrays = converter.get_input_arrays()
    for input_array in input_arrays:
        converter.quantizers[input_array].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.quantizers[layer_name].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
'''
quantize_layer_names2 = ['dense_1']
def convert_model_2(model,name='converted_quant_int_model2.tflite',quantize_annotations = quantize_layer_names2,partial=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    #converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = gen_rep()
    if partial:
        converter.quantize_annotations = quantize_annotations
    tflite_quant_int_model = converter.convert()
    #saving converted model in "converted_quant_model.tflite" file
    open(name, "wb").write(tflite_quant_int_model)

In [4]:
model=load_model("Yolov3.h5")
model.summary()
tt=just_convert_model_1(model)
#tflite_model = tf.keras.models.load_model(tt)
#tf.keras.models.save_model(tflite_model, 'model_quant.h5')


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, None, None, 3 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, None, None, 3 0           bnorm_0[0][0]                    
____________________________________________________________________________________________

ValueError: None is only supported in the 1st dimension. Tensor 'input_1' has invalid shape '[None, None, None, 3]'.

In [ ]:
tensorflow.__version__

In [ ]:
'''interpreter = tf.lite.Interpreter(model_path='j.tflite')
with open('j.tflite', 'rb') as f:
    tflite_model = f.read()

#tflite_model = tf.keras.models.load_model(tflite_model)'''

In [ ]:
#converter = tf.lite.TFLiteConverter.from_lite_model(interpreter.get_tensorflow_lite_model())
